Umgebung einrichten.
Hierzu ein How-to

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai
%pip install python-dotenv
%pip install translate

Verfügbarmachen des DEXPI-R&Is von GitHub

In [28]:
# Importieren des 'requests'-Moduls, um Daten von einer URL herunterzuladen, und das 'xml.etree.ElementTree'-Modul, um XML-Daten zu verarbeiten.
import requests
import xml.etree.ElementTree as ET

# GitHub Raw-URLs für die DEXPI-Dateie und die Formblatt-Vorlage.
github_url_dexpi = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/C01V01-HEX.EX03.xml'
github_url_formblatt = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/formblatt_6.2.xml'

# Herunterladen der DEXPI-Datei von der GitHub-URL.
response = requests.get(github_url_dexpi)

# Speichern der heruntergeladenen DEXPI-Datei lokal im Textmodus ('w').
with open('/tmp/C01V01-HEX.EX03.xml', 'w') as file:
    file.write(response.text)

# Definieren eines Dateipfades zur lokal gespeicherten DEXPI-Datei zur weiteren Verwendung.
dexpi_xml = "/tmp/C01V01-HEX.EX03.xml"

Prüfen der DEXPI-Datei

In [29]:
with open('/tmp/C01V01-HEX.EX03.xml', 'r') as file:
    dexpi_xml_content = file.read(1000)
    print(dexpi_xml_content)


<?xml version="1.0" encoding="utf-8"?>
<PlantModel xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="../ProteusPIDSchema_4.0.1.xsd">
  <!--Created with INGR ISO15926 PostProc V0.0.7.1 and ConfigurationFileVersion 1.1 by PPM-VM-2014-R1\Intergraph on PPM-VM-2014-R1 at 4/12/2018 5:14:33 PM-->
  <PlantInformation SchemaVersion="4.0.1" OriginatingSystem="SPPID" Date="2018-04-12" Time="15:13:38.0000000+02:00" Is3D="no" Units="Metre" Discipline="PID">
    <UnitsOfMeasure Distance="Metre" />
  </PlantInformation>
  <PlantStructureItem ID="PBS945900F29F424D32A292F07BB90792E2" ComponentClass="PlantSectionIso10209-2012" ComponentName="Github" ComponentClassURI="http://sandbox.dexpi.org/rdl/PlantSectionIso10209-2012">
    <GenericAttributes Number="2" Set="DexpiAttributes">
      <GenericAttribute Name="PlantSectionIdentificationCodeAssignmentClass" Value="Github" Format="string" AttributeURI="http://sandbox.dexpi.org/rdl/PlantSectionIdentificationCodeAssignment


Angaben zur Anlage

In [30]:

# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def lade_daten(dexpi_xml):
    try:
        # Lade das XML-Dokument und erhalte die Wurzel des Baums.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere die Anlagenbezeichnung aus dem XML-Baum.
        anlagenbezeichnung = root.find('.//Drawing').get('Name')
        # Weitere Werte können entsprechend extrahiert werden.

        # Gib ein Dictionary mit den extrahierten Werten zurück.
        return {
            'Anlagenbezeichnung': anlagenbezeichnung,
            # Weitere extrahierte Werte können hier hinzugefügt werden.
        }

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus und gibt None zurück.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")
        return None

# Beispielaufruf der Funktion und Ausgabe der extrahierten Daten.
daten = lade_daten(dexpi_xml)
print(daten)

{'Anlagenbezeichnung': 'C01V01-HEX.EX03'}


In [31]:
# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def lade_daten(dexpi_xml):
    try:
        # Lade das XML-Dokument und erhalte die Wurzel des Baums.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere ComponentClass aus allen Equipment- und PipingNetworkSystem-Elementen.
        equipment_elements = root.findall('.//Equipment')
        piping_network_system_elements = root.findall('.//PipingNetworkSystem')

        component_classes = set()

        for equipment_element in equipment_elements:
            component_class = equipment_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'PlateAndShellHeatExchanger', 'ShellAndTubeHeatExchanger']
            if component_class in erlaubte_component_classes:
                component_classes.add(component_class)

        for piping_network_system_element in piping_network_system_elements:
            component_class = piping_network_system_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['PipingNetworkSystem']
            if component_class in erlaubte_component_classes:
                component_classes.add(component_class)

        # Überprüfe, ob mindestens eine gültige ComponentClass gefunden wurde.
        if not component_classes:
            raise ValueError("Keine gültige ComponentClass gefunden.")

        # Weitere Werte können entsprechend extrahiert werden.

        # Gib ein Dictionary mit den extrahierten Werten zurück.
        return {
            'Anlagenumfang': list(component_classes),
            # Weitere extrahierte Werte können hier hinzugefügt werden.
        }

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus und gibt None zurück.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")
        return None

# Beispielaufruf der Funktion und Ausgabe der extrahierten Daten.
daten = lade_daten(dexpi_xml)
# print(daten)

def uebersetze_begriffe_manuell(begriffe, zielsprache='de'):
    manuelle_uebersetzungen = {
        'ReciprocatingPump': 'Kolbenpumpe',
        'PipingNetworkSystem': 'Rohrleitungssystem',
        'CentrifugalPump': 'Zentrifugalpumpe',
        'PlateAndShellHeatExchanger': 'Plattenwärmetauscher',
        'Tank': 'Behälter',
        'ShellAndTubeHeatExchanger': 'Rohrbündelwärmetauscher'
    }

    uebersetzte_begriffe = [manuelle_uebersetzungen.get(begriff, begriff) for begriff in begriffe]
    return uebersetzte_begriffe

# Beispielaufruf
komponentenklassen = ['ReciprocatingPump', 'PipingNetworkSystem', 'CentrifugalPump', 'PlateAndShellHeatExchanger', 'Tank', 'ShellAndTubeHeatExchanger']
uebersetzte_komponentenklassen = uebersetze_begriffe_manuell(komponentenklassen)

# Ausgabe der übersetzten Begriffe
print({'Anlagenumfang': uebersetzte_komponentenklassen})



{'Anlagenumfang': ['Kolbenpumpe', 'Rohrleitungssystem', 'Zentrifugalpumpe', 'Plattenwärmetauscher', 'Behälter', 'Rohrbündelwärmetauscher']}


In [32]:
def finde_groessten_tank(dexpi_xml):
    try:
        # Parsen der DEXPI-Datei
        root = ET.fromstring(dexpi_xml)
    except ET.ParseError as e:
        # Fehlerausgabe, wenn das Parsen fehlschlägt
        print("Fehler beim Parsen des XML-Dokuments:", e)
        return None, None
    
    # Initialisierung von Variablen für den größten Tank und dessen Kapazität
    groesster_tank = None
    max_capacity = 0
    
    # Iterieren über alle 'Equipment'-Elemente im XML-Dokument
    for tank in root.iter('Equipment'):
        if tank.attrib.get('ComponentClass') == 'Tank':
            # Suche nach dem Attribut 'NominalCapacity(Volume)' im aktuellen Tank
            capacity_attr = tank.find(".//GenericAttribute[@Name='NominalCapacity(Volume)']")
            
            # Überprüfen, ob das Attribut gefunden wurde und einen 'Value'-Schlüssel hat
            if capacity_attr is not None and 'Value' in capacity_attr.attrib:
                # Extrahieren der Kapazität und vergleichen mit der bisherigen maximalen Kapazität
                capacity = float(capacity_attr.attrib['Value'])
                if capacity > max_capacity:
                    max_capacity = capacity
                    groesster_tank = tank
    
    # Wenn ein größerer Tank gefunden wurde
    if groesster_tank is not None:
        # Extrahieren der Tag-Bezeichnung des größten Tanks
        tank_tag = groesster_tank.attrib['TagName']
        
        # Extrahieren der Beschreibungen der Kammern im größten Tank
        tank_beschreibungen = [chamber.find(".//GenericAttribute[@Name='ChamberDescriptionAssignmentClass']").attrib.get('Value', '') for chamber in groesster_tank.iter('Equipment') if chamber.attrib.get('ComponentClass') == 'Chamber']
        
        return groesster_tank, tank_tag, tank_beschreibungen
    else:
        # None, wenn kein größerer Tank gefunden wurde
        return None, None

# Aufruf mit XML-Dokument
with open(dexpi_xml, 'r') as file:
    dexpi_xml_content = file.read()

groesster_tank, tank_tag, tank_beschreibungen = finde_groessten_tank(dexpi_xml_content)

# Ausgabe der Ergebnisse
print("Größter Tank:", tank_tag)
print("Beschreibungen der Kammern:", tank_beschreibungen)

def setze_anlagenart(tank_beschreibungen):
    # Überprüfen, ob 'Processing Chamber' in den Beschreibungen vorkommt
    if "Processing Chamber" in tank_beschreibungen:
        return "HBV-Anlage"
    # Überprüfen, ob 'Storage Chamber' in den Beschreibungen vorkommt
    elif "Storage Chamber" in tank_beschreibungen:
        return "Tanklager"
    else:
        # Fehlermeldung, wenn keine oder beide Beschreibungen vorhanden sind
        print("Fehler: Ungültige Anlagenkonfiguration. Anlagenart manuell befüllen.")
        return None

# Beispielaufruf
anlagenart = setze_anlagenart(tank_beschreibungen)

# Ausgabe der Ergebnisse
print("Anlagenart:", anlagenart)



Größter Tank: T4750
Beschreibungen der Kammern: ['Processing Chamber', 'Jacket Chamber']
Anlagenart: HBV-Anlage


Angaben zu den wassergefährdenden Stoffen in der Anlage

Angaben zu Behältern

In [33]:
import xml.etree.ElementTree as ET

# Initialisierung der Variable außerhalb der Funktion
material_of_construction = None

def tank_suche(dexpi_xml_path):
    # Einlesen des Inhalts der DEXPI-XML-Datei
    with open(dexpi_xml_path, 'r') as file:
        dexpi_xml_content = file.read()

    # Parsen der dexpi_xml und Erstellen des Wurzelelements
    root = ET.fromstring(dexpi_xml_content)
    tanks = []

    # Durchlaufen aller Equipment-Elemente mit ComponentClass='Tank'
    for equipment in root.findall(".//Equipment[@ComponentClass='Tank']"):
        # Extrahieren des Tag-Namens des Tanks
        tag_name = equipment.get('TagName', '')

        # Aussortieren von Tanks ohne Tag-Namen, die mit 'T' beginnen
        if not tag_name.startswith('T'):
            continue

        # Extrahieren des NominalCapacity-Elements
        nominal_capacity_element = equipment.find(".//GenericAttribute[@Name='NominalCapacity(Volume)']")
        
        # Überprüfen, ob NominalCapacity-Element vorhanden ist
        if nominal_capacity_element is not None:
            # Extrahieren des Wertes von NominalCapacity
            nominal_capacity_str = nominal_capacity_element.get('Value', '')

            # Überprüfen, ob NominalCapacity eine Zahl ist
            try:
                nominal_capacity = float(nominal_capacity_str)
            except ValueError:
                # Fehlermeldung ausgeben und zum nächsten Tank gehen
                print(f"Fehler: NominalCapacity für Tank mit TagName '{tag_name}' ist keine Zahl.")
                continue
        else:
            # Fehlermeldung ausgeben und zum nächsten Tank gehen
            print(f"Fehler: Kein NominalCapacity-Attribut für Tank mit TagName '{tag_name}'.")
            continue

        # Extrahieren der Kammern des Tanks
        chambers = []
        for chamber in equipment.findall(".//Equipment[@ComponentClass='Chamber']"):
            chamber_function = chamber.find(".//GenericAttribute[@Name='ChamberFunctionAssignmentClass']").get('Value', '')
            
            # Nur Kammern mit 'Process Chamber' oder 'Storage Chamber' berücksichtigen
            if chamber_function in ['Processing', 'Storage']:
                material_of_construction = chamber.find(".//GenericAttribute[@Name='MaterialOfConstructionCodeAssignmentClass']").get('Value', '')
                chambers.append({
                    'Function': chamber_function,
                    'MaterialOfConstructionCode': material_of_construction
                })

        # Zusammenstellen der Tankdaten und Hinzufügen zur Liste
        tank_data = {
            'TagName': tag_name,
            'NominalCapacity': nominal_capacity,
            'Chambers': chambers
        }
        tanks.append(tank_data)

    # Rückgabe der Liste von Tanks
    return tanks

# Aufrufen der Funktion mit der DEXPI-XML-Datei und Ausgabe der Ergebnisse
tanks = tank_suche(dexpi_xml)

for tank in tanks:
    print(f'TagName: {tank["TagName"]}, NominalCapacity: {tank["NominalCapacity"]}')
    
    for chamber in tank['Chambers']:
        print(f'  Chamber Function: {chamber["Function"]}, MaterialOfConstructionCode: {chamber["MaterialOfConstructionCode"]}')





TagName: T4750, NominalCapacity: 22.0
  Chamber Function: Processing, MaterialOfConstructionCode: 1.4306


Quelle: https://python.langchain.com/docs/use_cases/chatbots/quickstart

In [ ]:
# Importieren der benötigten Module
import os

# Extrahieren des OpenAI-API-Schlüssels aus den Umgebungsvariablen oder direkt eingetragen
openai_api_key = os.getenv("OPENAI_API_KEY", "")

# Jetzt ist der OpenAI-API-Schlüssel verfügbar und kann im Code verwendet werden
#print(f"OpenAI API Key: {openai_api_key}")

# Importieren der benötigten Module
from langchain_openai import ChatOpenAI

# Initialisieren des ChatOpenAI-Objekts
chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2, openai_api_key=openai_api_key)


In [57]:
for tank in tanks:
    for chamber in tank['Chambers']:
        print(f'Behälternummer: {tank["TagName"]}, Nennvolumen (m3): {tank["NominalCapacity"]}, Material: {chamber["MaterialOfConstructionCode"]}')


from langchain_core.messages import HumanMessage

chat.invoke(
    [
        HumanMessage(
            content='Setze die Sprache auf Deutsch. Du bist ein Assistent im Bereich der Verfahrenstechnik. Du hast eine Aufgabe. Ich gebe dir eine Werkstoffbezeichnung. Diese Werkstoffbezeichnung kann unterschiedlich konkret sein. Deine Aufgabe besteht darin zuzuordnen, ob es sich um ein Kunststoff, ein Metall oder einen anderen Werkstoff handelt. Versuche also den gegebenen Werkstoff in eine der drei Kategorien: "Metall", "Kunststoff", "anderes Material" einzustufen. Gib mir als Ausgabe nur einen der drei Begriffe aus, ohne weitere Erklärung. Falls der gesuchte Werkstoff überhaupt nicht zuzuordnen ist, gib Error aus. Der gesuchte Werkstoff ist: 1.4306'
        )
    ]
)



Behälternummer: T4750, Nennvolumen (m3): 22.0, Material: 1.4306


AIMessage(content='Metall')

Angaben zu den Rohrleitungen

Back-Up

In [ ]:
# Später für Aufzählung
# Funktion zum Laden und Extrahieren von Daten aus der DEXPI-Datei.
def lade_daten(dexpi_xml):
    try:
        # Lade das XML-Dokument und erhalte die Wurzel des Baums.
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahiere die Anlagenbezeichnung aus dem XML-Baum.
        anlagenbezeichnung = root.find('.//Drawing').get('Name')

        # Extrahiere ComponentClass aus allen Equipment- und PipingNetworkSystem-Elementen.
        equipment_elements = root.findall('.//Equipment')
        piping_network_system_elements = root.findall('.//PipingNetworkSystem')

        component_classes = []

        for equipment_element in equipment_elements:
            component_class = equipment_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'PlateAndShellHeatExchanger', 'ShellAndTubeHeatExchanger']
            if component_class in erlaubte_component_classes:
                component_classes.append(component_class)

        for piping_network_system_element in piping_network_system_elements:
            component_class = piping_network_system_element.get('ComponentClass')

            # Überprüfe, ob die ComponentClass den erlaubten Werten entspricht.
            erlaubte_component_classes = ['PipingNetworkSystem']
            if component_class in erlaubte_component_classes:
                component_classes.append(component_class)

        # Überprüfe, ob mindestens eine gültige ComponentClass gefunden wurde.
        if not component_classes:
            raise ValueError("Keine gültige ComponentClass gefunden.")

        # Weitere Werte können entsprechend extrahiert werden.

        # Entferne Duplikate und erstelle ein eindeutiges Set von Komponentenklassen.
        unique_component_classes = set(component_classes)

        # Erstelle ein Dictionary mit dem Namen Anlagenumfang, in dem jede ComponentClass nur einmal vorkommt.
        Anlagenumfang = {klass: {} for klass in unique_component_classes}

        # Gib ein Dictionary mit den extrahierten Werten zurück.
        return {
            'anlagenbezeichnung': anlagenbezeichnung,
            'component_classes': component_classes,
            'Anlagenumfang': Anlagenumfang,
            # Weitere extrahierte Werte können hier hinzugefügt werden.
        }

    except Exception as e:
        # Falls ein Fehler auftritt, gib eine Fehlermeldung aus und gibt None zurück.
        print(f"Fehler beim Laden und Extrahieren der Daten: {e}")
        return None

# Beispielaufruf der Funktion und Ausgabe der extrahierten Daten.
daten = lade_daten(dexpi_xml)
print(daten)


{'anlagenbezeichnung': 'C01V01-HEX.EX03', 'component_classes': ['Tank', 'ReciprocatingPump', 'CentrifugalPump', 'ShellAndTubeHeatExchanger', 'PlateAndShellHeatExchanger', 'Tank', 'ReciprocatingPump', 'CentrifugalPump', 'ShellAndTubeHeatExchanger', 'PlateAndShellHeatExchanger', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem', 'PipingNetworkSystem'], 'Anlagenumfang': {'ShellAndTubeHeatExchanger': {}, 'PipingNetworkSystem': {}, 'CentrifugalPump': {}, 'ReciprocatingPump': {}, 'Tank': {}, 'PlateAndShellHeatExchanger': {}}}


In [ ]:
import requests
import xml.etree.ElementTree as ET

# GitHub Raw-URL für die DEXPI-R&I Datei
github_url_dexpi = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/C01V01-HEX.EX03.xml'
# GitHub Raw-URL für die Formblatt-Vorlage
github_url_formblatt = 'https://raw.githubusercontent.com/FloT10/MillerFlorinBA/main/formblatt_6.2.xml'

# Dateien lokal speichern
def lade_und_speichere(url, lokal_path):
    response = requests.get(url)
    with open(lokal_path, 'w') as file:
        file.write(response.text)

# Laden und Speichern der DEXPI-R&I Datei
dexpi_xml = '/tmp/C01V01-HEX.EX03.xml'
lade_und_speichere(github_url_dexpi, dexpi_xml)

# Laden und Speichern der Formblatt-Vorlage
formblatt_vorlage = '/tmp/formblatt_6.2.xml'
lade_und_speichere(github_url_formblatt, formblatt_vorlage)

# Funktion zum Laden und Extrahieren der Werte aus dem DEXPI-R&I
def lade_daten(dexpi_xml):
    try:
        # Laden des DEXPI-R&I Dokuments
        tree = ET.parse(dexpi_xml)
        root = tree.getroot()

        # Extrahieren der Werte, wie benötigt
        anlagenbezeichnung = root.find('.//Drawing').get('Name')
        # Weitere Werte entsprechend extrahieren

        # Kontrollausgabe der Anlagenbezeichnung
        print(f"Anlagenbezeichnung zur Kontrolle: {anlagenbezeichnung}")

        return {
            'anlagenbezeichnung': anlagenbezeichnung,
            # Weitere extrahierte Werte hinzufügen
        }

    except Exception as e:
        print(f"Fehler beim Laden und Extrahieren der Daten aus DEXPI-R&I: {e}")
        return None

# Funktion zum Befüllen der Formblatt-Vorlage mit den extrahierten DEXPI-R&I Daten
def befuellen_und_speichern(formblatt_vorlage, formblatt_befüllt, daten):
    try:
        tree = ET.parse(formblatt_vorlage)
        root = tree.getroot()

        # Befülle die Vorlage mit den extrahierten Daten
        root.find('.//anlagenbezeichnung').text = daten['anlagenbezeichnung']
        # Weitere Werte in der Vorlage entsprechend befüllen

        # Speichere die befüllte Vorlage als neue Datei
        tree.write(formblatt_befüllt)

        print(f"Das Formblatt wurde erfolgreich mit den Daten befüllt und unter {formblatt_befüllt} gespeichert.")

    except Exception as e:
        print(f"Fehler beim Befüllen und Speichern des Formblatts: {e}")

# Lade Daten aus der DEXPI-R&I Datei
daten_dexpi = lade_daten(dexpi_xml)

# Lade die Formblatt-Vorlage
formblatt_befüllt= '/tmp/Ziel/Formblatt_befüllt.xml'

# Befülle die Formblatt-Vorlage und speichere das Ergebnis
befuellen_und_speichern(formblatt_vorlage, formblatt_befüllt, daten_dexpi)


Anlagenbezeichnung zur Kontrolle: C01V01-HEX.EX03
Fehler beim Befüllen und Speichern des Formblatts: mismatched tag: line 244, column 2
